In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
pd.set_option('display.max_rows', 700)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
ipl = pd.read_csv('/kaggle/input/skillsay-ai-crickethon-dataset/matches.csv')

In [ ]:
ipl.info()

In [ ]:
ipl.head(5)

In [ ]:
ipl.shape

In [ ]:
ipl['season']=ipl['season'].astype('category')

In [ ]:
ipl.info()

Q) find the venue in which highest number of matches were held

In [ ]:
venue=ipl.groupby('venue')
venue.size().sort_values().reset_index().tail(1)

Q) Find the team winning most number of matches since 2008

In [ ]:
ipl['winner'].value_counts().sort_values().tail(1)

Q) Find the team winning least number of matches

In [ ]:
ipl['winner'].value_counts().sort_values().head(1).index[0]

Q) Find the team who played most number of matches. note down winning doesnt matter

In [ ]:
(ipl['team2'].value_counts() + ipl['team1'].value_counts()).sort_values().tail(1)

Q) List out all team names since 2008

In [ ]:
(ipl['team2'].value_counts() + ipl['team1'].value_counts()).drop_duplicates().index[::1]

In [ ]:
#plot bar graph for the teams and their number of matches played
(ipl['team2'].value_counts() + ipl['team1'].value_counts()).plot.bar()

Q) List out each seasons winners.
logic is that each seasons last match is the final match and its winner is the seasons winner

In [ ]:
x=ipl.drop_duplicates('season',keep='last')

In [ ]:
x[['season','winner']].sort_values('season')

Q) how many times each team has won the finals

In [ ]:
x['winner'].value_counts().reset_index()

In [ ]:
ipl.head(10)

Q)Find out whether the toss winner of the game has actually won the game by his decision to bat or field first?

In [ ]:
toss_data = ipl[['season','city','venue','toss_winner','toss_decision','winner']].copy()

In [ ]:
toss_data.head(10)

In [ ]:
toss_data['toss_with_bat']=""
toss_data['toss_with_field']=""

In [ ]:
d1 = toss_data['toss_winner']==toss_data['winner']

In [ ]:
toss_data = toss_data[d1].sort_values('toss_winner')

In [ ]:
def findValues_for_bat(toss_decision):
    if toss_decision=='bat':
        return 'yes'
    else:
        return 'no'
def findValues_for_field(toss_decision):
    if toss_decision=='field':
        return 'yes'
    else:
        return 'no'

In [ ]:
toss_data['toss_with_bat']=toss_data['toss_decision'].apply(findValues_for_bat)
toss_data['toss_with_field']=toss_data['toss_decision'].apply(findValues_for_field)

In [ ]:
toss_data.head(10)

In [ ]:
toss_data=toss_data.rename(columns={'toss_with_bat':'bat first','toss_with_field':'field first'})
toss_data.head(10)

In [ ]:
 t = toss_data.groupby(['season','toss_winner','bat first'],sort=True)
#t = toss_data.groupby(['season','toss_winner','toss_decision'],sort=True)

In [ ]:
t.size().head(15)

Q)Does the below thing have any real meaning? like is the below result useful?

In [ ]:
t.describe()

In [ ]:
delivery = pd.read_csv('/kaggle/input/skillsay-ai-crickethon-dataset/deliveries.csv')

In [ ]:
delivery.info()

In [ ]:
delivery.shape

In [ ]:
delivery.columns

In [ ]:
delivery.head(10)

Q) top 5 batsman in ipl according to most number of runs?

In [ ]:
t=delivery.groupby(['batsman'])['batsman_runs'].sum()

In [ ]:
t.sort_values(ascending=False).head(5)

Q) which batsman has played most number of balls

In [ ]:
t=delivery.groupby(['batsman'])

In [ ]:
t['batsman_runs'].count().sort_values(ascending=False).head(5) 
# logic is each row is one ball

Q) which batsman has hit highest number of 4s

In [ ]:
mask=delivery['batsman_runs']==4

In [ ]:
delivery_fours = delivery[mask].groupby('batsman')['batsman_runs'].count().sort_values(ascending=False)
# count() instead of sum() because we need to count the number of 4s instead of summing the total runs gained by hitting a boundary

In [ ]:
delivery_fours.head(5)

Q) how many total fours in ipl till now

In [ ]:
len(delivery[mask])

Q) which batsman has hit highest number of 6's

In [ ]:
mask=delivery['batsman_runs']==6
delivery_fours = delivery[mask].groupby('batsman')['batsman_runs'].count().sort_values(ascending=False)
delivery_fours.head(5)

Q) which bowler has given most no. of dot balls

In [ ]:
mask = delivery['total_runs']==0
delivery_dot = delivery[mask]
delivery_dot.groupby('bowler')['total_runs'].count().sort_values(ascending=False).head(5)

Q) batsman which when on non-striker end, there has been most dismissals

Case 1: Only the striker is dismissed everytime


Case 2: The striker and non-striker both can be dismissed

In [ ]:
#case 1
delivery_dismissal1 = delivery.dropna(subset=['player_dismissed'])
mask=delivery_dismissal1['batsman']==delivery_dismissal1['player_dismissed']
delivery_dismissal1 = delivery_dismissal1[mask]
ans=delivery_dismissal1.groupby('non_striker')['player_dismissed'].count().sort_values(ascending=False).head().reset_index()
#delivery_dismissal1.shape
ans

In [ ]:
delivery_dismissal2 = delivery.fillna({'player_dismissed':'Not'})
mask=delivery_dismissal2['player_dismissed']!='Not'
#mask1=delivery_dismissal2['player_dismissed']!=delivery_dismissal2['non_striker']
#delivery_dismissal2 = delivery_dismissal2[mask & mask1]
delivery_dismissal2 = delivery_dismissal2[mask]
ans1=delivery_dismissal2.groupby('non_striker')['player_dismissed'].count().sort_values(ascending=False).head(10).reset_index()
#delivery_dismissal2.shape
ans1

In [ ]:
#Below cell is the proof by showing that RG Sharma has been out being on non_strikers end just one time 
# Q:count the number of times batsman on strikers end got out while a particular batsman was on the non-strikers end.

In [ ]:
#Finding the cases when the non-striker was the player_dismissed
dell = delivery.dropna(subset=['player_dismissed'])
mask = dell['non_striker']==dell['player_dismissed']
mask1 = dell['non_striker']=='RG Sharma'
dell = dell[mask & mask1]
dell.groupby('non_striker')['player_dismissed'].count().sort_values(ascending=False).head(10).reset_index()

In [ ]:
7084+354 #non-striker can also get dismissed 

Q) batsman name and it returns the name of the team that batsman has hit most runs against

In [ ]:
# Finding the batsmans and the team they have hit most number of runs against
hit_runs = delivery.groupby(['bowling_team','batsman'])['batsman_runs'].sum().sort_values(ascending=False).reset_index().drop_duplicates(subset=['bowling_team'],keep='first')
hit_runs

In [ ]:
# Finding virat has hit most runs against which team
mask=delivery['batsman']=='V Kohli'
bats = delivery[mask]
bats.groupby('bowling_team')['batsman_runs'].sum().sort_values(ascending=False).index[0]

In [ ]:
def opp_team_most_runs(batsman_name):
    mask=delivery['batsman']==batsman_name
    bats = delivery[mask]
    ans=bats.groupby('bowling_team')['batsman_runs'].sum().sort_values(ascending=False).index[0]
    return ans

In [ ]:
opp_team_most_runs('RG Sharma')

Q)Batsman and it returns which bowler has given highest number of runs to that batsman

In [ ]:
mask=delivery['batsman']=='V Kohli'
bats = delivery[mask]
bats.groupby('bowler')['batsman_runs'].sum().sort_values(ascending=False).head()

In [ ]:
def most_runs_taken_from_bowler(batsman_name):
    mask=delivery['batsman']==batsman_name
    bats = delivery[mask]
    return bats.groupby('bowler')['batsman_runs'].sum().sort_values(ascending=False).index[0]

In [ ]:
most_runs_taken_from_bowler('V Kohli')

Q)in each over which team has hit how many sixes

In [ ]:
# Pivot_table method
sixes = delivery[delivery['batsman_runs']==6]
y=sixes.pivot_table(index=['batting_team'],columns=['over'],values='batsman_runs',aggfunc='count')
print(y)

In [ ]:
# Groupby method
sixes.groupby(['batting_team','over'])['batsman_runs'].count().head(10)

In [ ]:
import seaborn as sns

In [ ]:
sns.heatmap(y,cmap='summer')

Q) find the orange cap holders of each season

In [ ]:
merged_data = ipl.merge(delivery,left_on='id',right_on='match_id')

In [ ]:
merged_data.head()

In [ ]:
df = merged_data.groupby(['season','batsman'])['batsman_runs'].sum().reset_index()

In [ ]:
df.sort_values('batsman_runs').drop_duplicates(subset=['season'],keep='last').sort_values('season')

**Q1)** last 5 overs (death overs) find the dangerous batsman, strike rate (no. of runs divided by number of balls)*100.

base criteria is player has played 200 balls, (between 16 and 20 overs he has completed 200 balls)

In [ ]:
#only need to use data from 16 to 20 overs
df = delivery[delivery['over']>15] 

In [ ]:
#counting number of balls
a=df.groupby('batsman')['batsman_runs'].count() 

In [ ]:
# only need to take players who played more than 200 balls
b=df.groupby('batsman')['batsman_runs'].count()>200 

In [ ]:
c=a[b].index.tolist()

In [ ]:
c

In [ ]:
df.head(10)

In [ ]:
#isin() function is used to find only those rows in which batsman names matches those names..
# stored in a list in c, this 
d=df[df['batsman'].isin(c)] 

In [ ]:
runs=d.groupby('batsman')['batsman_runs'].sum()

In [ ]:
balls=d.groupby('batsman')['batsman_runs'].count()

In [ ]:
(runs/balls*100).sort_values(ascending=False)

**Q2)** top 10 batsman, and top 10 bowlers with max number of wickets and combine them and make a heatmap which tells which top batsman has hitten most runs against a top bowler

In [ ]:
#Finding the top 10 batsman and storing the names in a list using index.tolist() function

top_batsman=delivery.groupby('batsman')['batsman_runs'].sum().sort_values(ascending=False).head(10).index.tolist()

In [ ]:
top_batsman

In [ ]:
delivery['dismissal_kind'].value_counts()

In [ ]:
#only in these cases the wicket credit is given to bowler
dismissal=['caught','bowled','lbw','stumped','caught and bowled','hit wicket']

In [ ]:
#Filtering out batsman who got out by the above ways
out=delivery[delivery['dismissal_kind'].isin(dismissal)]

In [ ]:
#Taking out list of top 10 bowlers having most wickets
bowler=out.groupby('bowler')['dismissal_kind'].count().sort_values(ascending=False).head(10).index.tolist()

In [ ]:
len(out)

In [ ]:
batsmandf=delivery[delivery['batsman'].isin(top_batsman)]
# fitlering out top10 batsman and storing in new variable 

In [ ]:
finaldf=batsmandf[batsmandf['bowler'].isin(bowler)]
# filtering out batsmans who faced the top10 batsmans

In [ ]:
finaldf.shape

In [ ]:
y=finaldf.pivot_table(index='bowler',columns='batsman',values='batsman_runs',aggfunc='sum')

In [ ]:
sns.heatmap(y,cmap='summer')

**Q3)** most economical bowler in death overs

Economy rate = runs conceeded / overs bowled

Example if a bowler has given 35 runs in 3.1 overs

so his overs bowled will be calculated as 3+1/6 = 3.1666

and Economy rate would be 35/3.166 = 11.054

In [ ]:
do=delivery[delivery['over']>15]
#we only need overs between 16 to 20 overs

In [ ]:
len(do)

In [ ]:
a=do.groupby('bowler')['total_runs'].count() #Finding number of balls bowled by each bowler
a.sort_values(ascending=False)

In [ ]:
b=do.groupby('bowler')['total_runs'].count()>100 #Only need bowlers who bowled more than 100 balls

In [ ]:
bowler=a[b].index.tolist() # storing bowler names in a list

In [ ]:
newdf=delivery[delivery['bowler'].isin(bowler)]

In [ ]:
newdf.shape

In [ ]:
run=newdf.groupby('bowler')['total_runs'].sum()

In [ ]:
balls=newdf.groupby('bowler')['total_runs'].count()

In [ ]:
balls=balls/6

In [ ]:
(run/balls).sort_values().head